In [52]:
import sys, os
from pymongo import MongoClient

from plotly.offline import init_notebook_mode, iplot
from plotly.graph_objs import *

# https://stackoverflow.com/questions/41323423/plotly-inside-jupyter-notebook-python
init_notebook_mode(connected=True) # initiate notebook for offline plot

import pandas as pd

In [53]:
sys.path.append("..")
import config # ../config.py

# run on raspberrypi3 Mongo database
#config.mongodb_ip = "192.168.1.224"
# run on Asus laptop Mongo database
#config.mongodb_ip = "192.168.1.124"

client = MongoClient(host=config.mongodb_ip, port=config.mongodb_port)
db = client['steam']

## Failed appids excluded from future attempts or analysis

In [54]:
failed_appids_df = pd.DataFrame(list(db['apps'].find({"failureCount": {"$gte":3}}, {"appid":1,"name":1,"_id":False})))
failed_appids_df

,appid,name
0,12630,Legend: Hand of God
1,15310,The Settlers: Heritage of Kings
2,19930,The Settlers: Rise of an Empire Gold Edition
3,23330,The Last Remnant Demo
4,33910,Arma 2
5,33220,Tom Clancy's Splinter Cell: Conviction
6,63600,realMyst
7,81675,Ticket to Ride Trailer DE
8,81674,Ticket to Ride Trailer EN
9,81676,Ticket to Ride Trailer FR


In [55]:
grouped_df = db['apps'].aggregate([
{"$match":
    {"$or": [{"failureCount": {"$lt": 3}}, {"failureCount": {"$exists": False}}]}
},
{"$group":  {"_id": {"$dateToString": {"format": "%Y-%m-%d", "date": "$updated_date"}},
             "count" : { "$sum" : 1} } },
{"$sort": {"_id": 1}}
])

grouped_df = pd.DataFrame(list(grouped_df))
grouped_df.rename(columns={grouped_df.columns[0] : 'updated_date'}, inplace=True)
grouped_df

,updated_date,count
0,2019-04-15,436
1,2019-04-16,3587
2,2019-04-17,3478
3,2019-04-18,3537
4,2019-04-19,3780
5,2019-04-20,1749
6,2019-04-21,2993
7,2019-04-22,3003
8,2019-04-23,3311
9,2019-04-24,3553


## Large numbers of old AppID entries means `refreshsteam.py` needs to be run.  This excludes consistently failed appids.

In [56]:
# https://plot.ly/python/bar-charts/

data = [Bar(
        x=grouped_df['updated_date'],
        y=grouped_df['count']
    )]

layout = Layout(
        title='AppIDs By Last Updated Date',
        xaxis=dict(
            title='Date'
        ),
        yaxis=dict(
            title='Count'
        )
    )

fig = Figure(data=data, layout=layout)

iplot(fig)

In [57]:
grouped_df = db['pricehistory'].aggregate([
{"$match": {}},
{"$group":  {"_id": {"$dateToString": {"format": "%Y-%m-%d", "date": "$date"}},
             "count" : { "$sum" : 1} } },
{"$sort": {"_id": 1}}
])

grouped_df = pd.DataFrame(list(grouped_df))
grouped_df.rename(columns={grouped_df.columns[0] : 'date'}, inplace=True)
grouped_df

,date,count
0,2018-09-04,8215
1,2018-09-05,43441
2,2018-09-06,37421
3,2018-09-07,23655
4,2018-09-08,37188
5,2018-09-10,2963
6,2018-09-11,45999
7,2018-09-12,20210
8,2018-09-13,269
9,2018-09-14,37390


## This is historical data from the `pricehistory` Mongo collection.  This is run roughly every 48 hours.

In [58]:
# https://plot.ly/python/bar-charts/

data = [Bar(
        x=grouped_df['date'],
        y=grouped_df['count']
    )]

layout = Layout(
        title='Distribution of Price History',
        xaxis=dict(
            title='Date'
        ),
        yaxis=dict(
            title='Records Count'
        )
    )

fig = Figure(data=data, layout=layout)

iplot(fig)

In [59]:
grouped_df = db['steamusers'].aggregate([
{"$match": {}},
{"$group":  {"_id": {"$dateToString": {"format": "%Y-%m-%d", "date": "$date"}},
             "count" : { "$sum" : 1} } },
{"$sort": {"_id": 1}}
])

grouped_df = pd.DataFrame(list(grouped_df))
grouped_df.rename(columns={grouped_df.columns[0] : 'date'}, inplace=True)
grouped_df

,date,count
0,2018-12-19,119
1,2018-12-20,340
2,2018-12-21,751
3,2018-12-22,1021
4,2018-12-23,932
5,2018-12-24,980
6,2018-12-25,938
7,2018-12-26,924
8,2018-12-27,1106
9,2018-12-28,1287


## These are the number of records we have per day for the `steamusers` Mongo collection.  This is created using the `steamusers.py` script.

In [60]:
# https://plot.ly/python/bar-charts/

data = [Bar(
        x=grouped_df['date'],
        y=grouped_df['count']
    )]

layout = Layout(
        title='Distribution of Records Per Day for steamusers',
        xaxis=dict(
            title='Date'
        ),
        yaxis=dict(
            title='Records Count'
        )
    )

fig = Figure(data=data, layout=layout)

iplot(fig)

In [61]:
grouped_df = db['topgames'].aggregate([
{"$match": {}},
{"$group":  {"_id": {"$dateToString": {"format": "%Y-%m-%d", "date": "$date"}},
             "count" : { "$sum" : 1} } },
{"$sort": {"_id": 1}}
])

grouped_df = pd.DataFrame(list(grouped_df))
grouped_df.rename(columns={grouped_df.columns[0] : 'date'}, inplace=True)
grouped_df

,date,count
0,2019-01-08,300
1,2019-01-09,2300
2,2019-01-10,100
3,2019-01-13,4000
4,2019-01-14,9000
5,2019-01-15,8400
6,2019-01-16,9200
7,2019-01-17,8500
8,2019-01-18,9100
9,2019-01-19,5200


## These are the number of entries we have per day for the `topgames` Mongo collection.  Each query returns the current top 100 games being played.  This is run every 15 minutes.

In [62]:
# https://plot.ly/python/bar-charts/

data = [Bar(
        x=grouped_df['date'],
        y=grouped_df['count']
    )]

layout = Layout(
        title='Distribution of Records Per Day for topgames',
        xaxis=dict(
            title='Date'
        ),
        yaxis=dict(
            title='Records Count'
        )
    )

fig = Figure(data=data, layout=layout)

iplot(fig)

## Storage and disk usage information

In [63]:
# https://stackoverflow.com/questions/1094841/reusable-library-to-get-human-readable-version-of-file-size
def sizeof_fmt(num, suffix='B'):
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f %s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f %s%s" % (num, 'Yi', suffix)

for collection in db.list_collection_names():
    stats = db.command("collstats", str(collection))
    print(collection + " size: " + sizeof_fmt(stats['size']))

print("--------------------")
stats = db.command("dbstats")
print("Total Steam DB Size: " + sizeof_fmt(stats['dataSize']))

topgames size: 159.0 MiB
apps size: 519.3 MiB
steamusers size: 6.4 MiB
pricehistory size: 389.0 MiB
--------------------
Total Steam DB Size: 1.0 GiB
